In [3]:
from tokenizers import Tokenizer
from fourm.vq.vqvae import VQVAE, DiVAE
import os
import matplotlib.pyplot as plt
from PIL import Image
import scipy.io
import cv2
from IPython.display import display, clear_output
import time
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import requests
from transformers import BlipProcessor, BlipForConditionalGeneration
import numpy as np
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
from fourm.data.modality_transforms import RGBTransform
from fourm.utils import denormalize, IMAGENET_INCEPTION_MEAN, IMAGENET_INCEPTION_STD, IMAGENET_DEFAULT_STD, IMAGENET_DEFAULT_MEAN
# The flag below controls whether to allow TF32 on matmul. This flag defaults to False in PyTorch 1.12 and later.
torch.backends.cuda.matmul.allow_tf32 = True
# The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
torch.backends.cudnn.allow_tf32 = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_grad_enabled(False)

In [4]:
from huggingface_hub import snapshot_download
from cosmos_tokenizer.image_lib import ImageTokenizer
snapshot_download(
    repo_id='nvidia/Cosmos-0.1-Tokenizer-DI16x16', 
    local_dir='/tmp/nvidiaxxxx/Cosmos-0.1-Tokenizer-DI16x16'
)
image_tokenizer = ImageTokenizer(
    checkpoint_enc='/tmp/nvidiaxxxx/Cosmos-0.1-Tokenizer-DI16x16/encoder.jit',
    checkpoint_dec='/tmp/nvidiaxxxx/Cosmos-0.1-Tokenizer-DI16x16/decoder.jit',
).to(device)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:


processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [6]:
img = Image.open('penn_action_raw/Penn_Action/frames/0001/000001.jpg').resize((640,640))
img_tensor = TF.to_tensor(img).to(device).unsqueeze(0) * 2 - 1
print(img_tensor.shape)
inputs = processor(images=img, return_tensors="pt")

# Generate caption
out = model.generate(**inputs)
caption = processor.decode(out[0], skip_special_tokens=True)
print(caption)
print(out)

torch.Size([1, 3, 640, 640])
a man standing on a tennis court holding a tennis racket
tensor([[30522,  1037,  2158,  3061,  2006,  1037,  5093,  2457,  3173,  1037,
          5093, 14513,  3388,   102]])


In [7]:
print(out.shape)

torch.Size([1, 14])
